In [ ]:
#Installs:
!pip3 install torch
!pip3 install pandas
!pip3 install torchvision

In [1]:
import torch
torch.cuda.is_available()


True

In [2]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms

labels_dict_float = {
    'cutie_carton': torch.tensor([1.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    'gunoi_menajer': torch.tensor([0.0, 1.0, 0.0, 0.0, 0.0, 0.0]),
    'hartie_carton': torch.tensor([0.0, 0.0, 1.0, 0.0, 0.0, 0.0]),
    'metal': torch.tensor([0.0, 0.0, 0.0, 1.0, 0.0, 0.0]),
    'sticla': torch.tensor([0.0, 0.0, 0.0, 0.0, 1.0, 0.0]),
    'sticla_plastic': torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
}



class CustomImageDataset():
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):

        files = os.listdir(self.img_dir)
        return len(files)


    def __getitem__(self, idx):
        
        files = os.listdir(self.img_dir)
        img_path = os.path.join(self.img_dir, files[idx])        
        image = Image.open(img_path).convert("RGB")
        resized_img = image.resize((224, 224))

        transform = transforms.ToTensor()
        final_img = transform(resized_img)

        
        no_extension = files[idx].split('.', 1)[0]
        no_numbers = ''.join([i for i in no_extension if not i.isdigit()])
        label = no_numbers[:len(no_numbers)-1]
        tensor_label = labels_dict_float[label]
        
        if self.transform:
            image = self.transform(final_img)
        if self.target_transform:
            label = self.target_transform(tensor_label)
        return final_img, tensor_label



In [7]:
from torchvision import transforms
batch_size = 16
train = DataLoader(CustomImageDataset("/home/alex/Documents/hackitall/dataset/data"), batch_size=batch_size)
train_features, train_labels = next(iter(train))
print(train_features.shape)
print(train_labels)

torch.Size([16, 3, 224, 224])
tensor([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])


In [4]:
from torchvision import models
net = models.vgg16(pretrained = True)

/home/alex/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/alex/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
import torch.nn as nn
net.classifier[6] = nn.Linear(in_features=4096, out_features=6)
net.train()

In [ ]:
    for param in net.parameters():
        param.requires_grad = False

    for param in net.classifier.parameters():
        param.requires_grad = True


In [ ]:
import torch.optim as optim
cuda = torch.device('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.classifier.parameters(), lr=1e-3, momentum=0.9)

net.to(cuda)
criterion.to(cuda)


In [ ]:
# training function
import datetime

def train_model(net, criterion, optimizer, num_epochs):
    
    # train = DataLoader(CustomImageDataset, batch_size = 32)
    # cuda = torch.device('cuda')

    for epoch in range (num_epochs):
        for img, labels in train:
            
            img, labels = img.to(cuda), labels.to(cuda)

            optimizer.zero_grad()
            outputs = net(img)

            loss = criterion(outputs, torch.tensor(labels))
        
            loss.backward()

            optimizer.step()
        print(epoch, datetime.datetime.now())

In [ ]:
import gc
num_epochs = 50

gc.collect()
torch.cuda.empty_cache()

train_model(net, criterion, optimizer, num_epochs)

In [ ]:
torch.save(net.state_dict(), "finetuned-vgg16.pth")

In [ ]:
imgs, labels = next(iter(train))

In [ ]:
net(imgs.to(cuda)).detach()

In [1]:
import torch
from torchvision import models
import torch.nn as nn

net = models.vgg16(pretrained = False)
net.classifier[6] = nn.Linear(in_features=4096, out_features = 6)

net.load_state_dict(torch.load('finetuned-vgg16.pth'))

/home/alex/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/alex/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [14]:
yhat = net(train_features).detach()
yhat.argmax(axis = 1)


tensor([4, 4, 0, 4, 5, 2, 0, 3, 2, 3, 0, 3, 3, 0, 4, 5])

In [15]:
print(train_labels)

tensor([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])


In [27]:
t = yhat.argmax(axis=1)

dict = {
    0: 'Cutie de carton',
    1 : 'Gunoi menajer',
    2 : 'Hartie/Carton',
    3 : 'Metal',
    4 : 'Sticla',
    5 : 'Sticla de Plastic'
}

for i in t:
    print(dict[i.item()])


Sticla
Sticla
Cutie de carton
Sticla
Sticla de Plastic
Hartie/Carton
Cutie de carton
Metal
Hartie/Carton
Metal
Cutie de carton
Metal
Metal
Cutie de carton
Sticla
Sticla de Plastic


In [4]:
img_path = "/home/alex/Documents/hackitall/image.jpeg"
image = Image.open(img_path).convert("RGB")
resized_img = image.resize((224, 224))

convert_tensor = transforms.ToTensor()
tensor_image = convert_tensor(resized_img)
x = torch.unsqueeze(tensor_image, dim = 0)
yhat = net(x).detach()
i = yhat.argmax(axis = 1)

dict = {
    0: 'Cutie de carton',
    1 : 'Gunoi menajer',
    2 : 'Hartie/Carton',
    3 : 'Metal',
    4 : 'Sticla',
    5 : 'Sticla de Plastic'
}
print(dict[i.item()])


Sticla de Plastic
